In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('No GPU available')
else:
    print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

# Elliptic Exploration

In [1]:
import os
import sys

import pathlib

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# supervised baseline
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# visualization
import scikitplot as skplt


# feature/model selection
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV

# evaluation
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve, average_precision_score, precision_recall_fscore_support

import logging
logging.basicConfig(stream=sys.stdout, format='',
                level=logging.INFO, datefmt=None)
logger = logging.getLogger('elliptic_scouting')

from IPython.display import display, Markdown, HTML, Image

sys.path.append('../')
from utils import *
from evaluation.model_performance import *

In [3]:
import torch

In [ ]:
df_classes, df_edge, df_features = import_elliptic_data_from_csvs()

In [ ]:
df_classes.info()

In [ ]:
df_edge.info()

In [ ]:
df_features.info()

In [ ]:
df_features.head()

In [ ]:
df_features = rename_features(df_features)
df_classes = df_classes.replace({'2': 'licit', '1': 'illicit'})

In [ ]:
sns.countplot(x=df_classes['class'])

In [ ]:
df_features['time_step'].value_counts().sort_index().plot()
plt.title('Number of transactions (licit/illicit) for timestep')

In [ ]:
df_features_class = pd.merge(df_features, df_classes, left_on='id', right_on='txId', how='left')
df_features_class = df_features_class.drop(columns='txId')
df_features_class.head()

In [ ]:
plt.figure(figsize=(12, 8))
grouped = df_features_class.groupby(['time_step', 'class'])['id'].count().reset_index().rename(columns={'id': 'count'})
sns.lineplot(x='time_step', y='count', hue='class', data=grouped)
plt.legend(loc=(1.0, 0.8))
plt.title('Number of transactions in each time step by class')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
grouped = df_features_class.groupby(['time_step', 'class'])['trans_feat_0'].mean().reset_index().rename(columns={'trans_feat_0': 'mean'})
sns.lineplot(x='time_step', y='mean', hue='class', data=grouped);
plt.legend(loc=(1.0, 0.8))
plt.title('Trans Feat 0 mean for timestep and classes')
plt.show()

In [ ]:
licit_mask = df_features_class['class'] == 'licit'
illicit_mask = df_features_class['class'] == 'illicit'
mask_known = licit_mask | illicit_mask
mask_known.sum()

In [ ]:
df_known = df_features_class[mask_known].copy()
df_known = df_known.replace({'licit': 0, 'illicit': 1})
df_known.head()

In [ ]:
g = df_known.groupby(['time_step', 'class'])['agg_feat_2'].mean().reset_index().rename(columns={'agg_feat_2': 'mean'})
sns.lineplot(x='time_step', y='mean', hue='class', data=g)
plt.legend(loc=(1.0, 0.8))
plt.title('agg_feat_2 for licit/illicit classes')
plt.show()

### Features selection

In [ ]:
last_time_step = 49
last_train_time_step = 34
only_labeled = True

X_train_df, X_test_df, y_train, y_test = run_elliptic_preprocessing_pipeline(last_train_time_step=last_train_time_step,
                                                                             last_time_step=last_time_step,
                                                                             only_labeled=only_labeled)

In [ ]:
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train_df.values)
pca = PCA(n_components=166, whiten=True, random_state=0)

In [ ]:
pca.fit(X_train_sc)
var = np.cumsum(np.round(pca.explained_variance_ratio_, decimals=3)*100)
var  # cumulative sum of variance explained with [n] features

In [ ]:
plt.ylabel('% Variance Explained')
plt.xlabel('# of Features')
plt.title('PCA Analysis')
plt.ylim(30,100.5)
plt.style.context('seaborn-whitegrid')
plt.plot(var)

In [ ]:
import random 

def supervised_model_cv_fit_predict(X_train_df, y_train, X_test_df, model, runs=5):
  y_preds = []
  for i in range(runs):
    model.fit(X_train_df, y_train)
    y_pred = model.predict(X_test_df)
    y_preds.append(y_pred)
  return y_preds
  
  
def plot_precision_recall_roc(y_test, y_probs):
    
    scores = {'avg_precision': [], 'best_f1': []}
    for y_prob in y_probs:
      precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
      f1_scores = 2*recall*precision/(recall+precision)
      #TODO: by removing nan f1-scores thresholds indexes are no more aligned
      f1_scores = f1_scores[~np.isnan(f1_scores)]
    
      best_threshold = thresholds[np.argmax(f1_scores)]
      display(Markdown(f'Best threshold: {best_threshold:.3f}'))
      best_f1_score = np.max(f1_scores)
      scores['best_f1'].append(best_f1_score)
      display(Markdown(f'Best F1-Score: {best_f1_score:.3f}'))

      average_precision = average_precision_score(y_test, y_prob)
      scores['avg_precision'].append(average_precision)
      display(Markdown(f'Average precision: {average_precision:.3f}'))
      display(Markdown(f'Percentage of true labels: {y_test.sum()/len(y_test):.3f}'))

      probas = np.column_stack((1 - y_prob, y_prob))
      skplt.metrics.plot_precision_recall(y_test, probas)
      plt.show()
      skplt.metrics.plot_roc(y_test, probas)
      plt.show()
    
    avg_precision = np.mean([score for score in scores['avg_precision']])
    avg_best_f1 = np.mean([score for score in scores['best_f1']])
    return avg_precision, avg_best_f1

def plot_confusion_matrix(y_true, y_preds, title=None, xtickslabels=None, ytickslabels=None):

    for y_pred in y_preds:
      precision, recall, f1score, support = precision_recall_fscore_support(y_true, y_pred)
      display(Markdown(f'Precision {precision}, recall {recall}, f1score {f1score}, support {support}'))    
    
      ax = skplt.metrics.plot_confusion_matrix(
          y_true,
          y_pred,
          normalize=True,
          figsize=(10, 8),
          title=title
         )
    
      if xtickslabels is not None:
        ax.set_xticklabels(xtickslabels)

      if ytickslabels is not None:
        ax.set_yticklabels(ytickslabels)
        
      plt.show()

## Supervision models

In [ ]:
X_train = X_train_df.values
X_test = X_test_df.values

In [ ]:
y_test.value_counts(dropna=False)

In [ ]:
sns.countplot(x=y_test)

### Logistic Regression

In [ ]:
y_preds_lr = supervised_model_cv_fit_predict(X_train, y_train, X_test, LogisticRegression(max_iter=10000))

In [ ]:
display(Markdown('LR classifier'))
plot_precision_recall_roc(y_test, y_preds_lr)

In [ ]:
plot_confusion_matrix(y_test, y_preds_lr, title='LR')

### XGBoost

In [ ]:
y_preds_xgboost = supervised_model_cv_fit_predict(X_train, y_train, X_test, XGBClassifier())

In [ ]:
display(Markdown('XGB classifier'))
plot_precision_recall_roc(y_test, y_preds_xgboost)

In [ ]:
plot_confusion_matrix(y_test, y_preds_xgboost, title='XGBoost')

### Random Forest

In [ ]:
y_preds_rf = supervised_model_cv_fit_predict(X_train, y_train, X_test, RandomForestClassifier())

In [ ]:
display('RF classifier')
plot_precision_recall_roc(y_test, y_preds_rf)

In [ ]:
plot_confusion_matrix(y_test, y_preds_rf, title='RF')

### LightGBM

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from multiprocessing import cpu_count

n_cpu = cpu_count()

params = {
      'nthread': n_cpu,
      'objective': 'binary',
      'metric': 'cross-entropy',
      'learning_rate': 0.05,
      'num_leaves': 63,
      'min_data_in_leaf': 10,
      'feature_fraction': 0.8,
      'bagging_fraction': 0.8,
      'bagging_freq': 5,
      'reg_alpha': 1,
      'reg_lambda': 1,
      'verbose': 1,
      'verbose_eval': True,
    }

def train_model(X, y, weight, parameters, num_boost):
    logger.info(f'Training model: weight {weight}')
    logger.info(f'parameters {parameters}, num_boost {num_boost}')
    
    train_data = lgb.Dataset(X, label=y, weight=weight)
    model = lgb.train(parameters, train_data, num_boost_round=num_boost)
    logger.info(f'model trained: ')
    return model
  
def get_balanced_weights(labels):
    class_weight = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
    weights = labels.map(lambda x: class_weight[0] if x == False else class_weight[1])
    return weights

In [ ]:
model = train_model(X_train, y_train, get_balanced_weights(y_train), params, 5)

In [ ]:
SPLIT_WEIGHT = 0.6

importance_split = model.feature_importance(importance_type='split')
importance_gain = model.feature_importance(importance_type='gain')

importance = SPLIT_WEIGHT * importance_split / importance_split.sum() + \
             (1 - SPLIT_WEIGHT) * importance_gain / importance_gain.sum()

importance = pd.Series((100 * importance).round(decimals=1), index=model.feature_name())
importance = importance[importance > 0].dropna()


with sns.axes_style('dark'):
    sns.set(font_scale=1.1)
    ax = importance.plot.barh(figsize=(10, 35), title='Feature Importance');

    for p in ax.patches:
        width = p.get_width()
        _ = ax.text(width, p.get_y(), f'{width}%')

In [ ]:
lgb_clf = lgb.sklearn.LGBMClassifier(n_jobs=n_cpu)
y_preds_lgb = supervised_model_cv_fit_predict(X_train_df, y_train, X_test, lgb_clf)

## Show aggregate results for timestemp

In [ ]:
def calc_avg_score_and_std_per_timestep(X_test_df, y_test, y_preds, aggregated_timestamp_column='time_step', scoring= 'f1'):
    last_train_time_step = min(X_test_df['time_step']) - 1
    last_time_step = max(X_test_df['time_step'])
    all_model_scores = []
    for y_pred in y_preds:
        model_scores = []
        for time_step in range(last_train_time_step + 1, last_time_step + 1):
            time_step_idx = np.flatnonzero(X_test_df[aggregated_timestamp_column] == time_step)
            y_true_ts = y_test.iloc[time_step_idx]
            y_pred_ts = [y_pred[i] for i in time_step_idx]
            model_scores.append(calculate_model_score(y_true_ts.astype('int'), y_pred_ts, scoring))
        all_model_scores.append(model_scores)

    avg_f1 = np.array([np.mean([f1_scores[i] for f1_scores in all_model_scores]) for i in range(15)])
    std = np.array([np.std([f1_scores[i] for f1_scores in all_model_scores]) for i in range(15)])

    return avg_f1, 

In [ ]:
avg_f1_lr_ts = calc_avg_score_and_std_per_timestep(X_test_df, y_test, y_preds_lr)
avg_f1_rf_ts  = calc_avg_score_and_std_per_timestep(X_test_df, y_test, y_preds_rf)
avg_f1_xgboost = calc_avg_score_and_std_per_timestep(X_test_df, y_test, y_preds_xgboost)
avg_f1_lgb_ts = calc_avg_score_and_std_per_timestep(X_test_df, y_test, y_preds_lgb)
model_f1_ts_dict = {'LR': avg_f1_lr_ts,
                    'XGB': avg_f1_xgboost, 
                    'Random Forest': avg_f1_rf_ts, 
                    'LightGBM': avg_f1_lgb_ts}

In [ ]:
from evaluation import plot_evaluation as pe
import importlib
importlib.reload(pe)

In [ ]:
plt.style.use('grayscale')
pe.plot_performance_per_timestep(model_metric_dict=model_f1_ts_dict, last_train_time_step=last_train_time_step,
                              last_time_step=last_time_step, linewidth=1.5, figsize=(10, 10), labelsize=20, fontsize=22,
                              markers=['P', '^', 'o', 's'],
                              linestyles=['-', ':', 'dashed', 'dashdot'],
                              linecolor=['pink', 'orange', 'firebrick', 'forestgreen'],
                              barcolor='royalblue', baralpha=0.3)